# For the Lohse dataset at
http://web1.genepool.private/runinfo/hesiod_reports/20191107_EGS1_11921LK0002/all_reports/report.3cells.pan.html

In [ ]:
%matplotlib inline

import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# We can load the datasets from the .pkl files. Also use a convenience function from
# plot_lambda_identity_by_time.py. If this fails, check the gvfs is mounted.
from plot_lambda_identity_by_time import gen_bins

# Give me girth!
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
sns.set(rc={'figure.figsize':(16.0,8.0)})

# And a decent palette
pal = sns.cubehelix_palette(12, rot=-.5, dark=.3)


In [ ]:
# Read the pickle file made by plot_lambda_identity_by_time.py
df = pd.read_pickle("gseg_link/eb80ac83.pkl")

# Add bins as in the script (since these don't get pickled)
max_time = math.ceil(df['StartTime'].max())
bin_size, bin_labels = gen_bins(1800, 16, max_time)

cut_range = list(range(0, max_time + bin_size, bin_size))[:len(bin_labels) + 1]
df['ReadTime'] = pd.cut(df['StartTime'], cut_range, right=False, labels=bin_labels)

# Also add this which will be useful later
df['UnalignedLength'] = df['ReadLength'] - df['AlignmentLength']

df.head()


In [ ]:
# Basic plot of Accuracy over time
# Remember that Accuracy is AS / ReadLength but AlignmentAccuracy is AS / AlignementLength

sns.boxplot(data=df, x='ReadTime', y='Accuracy', palette=pal)

In [ ]:
sns.boxplot(data=df, x='ReadTime', y='AlignmentScore', fliersize=2, palette=pal)
sns.swarmplot(data=df.sample(n=2000), x='ReadTime', y='AlignmentScore')

In [ ]:
# Those funny outliers at the top - a longer section of lambda?! Need to pop them in an alignment viewer.
df[df['AlignmentScore'] > 4000]

In [ ]:
# Those reads at the end when most pores were blocked
df[df.ReadTime == '12:00']

## Now we focus on just the reads at time point 01:00

In [ ]:
df_1 = df[df.ReadTime == '01:00']
df_1_short = df_1[df_1.ReadLength < 1000]
df_1_short.head()

In [ ]:
# Add a categorical hue column to the frame. We'll use this in the scatter plots.
# I got these numbers by eyeballing the graphs not from any real source.

df['my_hue'] = np.select([df.UnalignedLength > 250, 
                                (3340 < df.ReadLength) & (df.ReadLength < 3620)],
                           ['lowmatch',
                                'good'],
                                   default="short")

df_1 = df[df.ReadTime == '01:00']
df_1.my_hue.value_counts().plot(kind="pie", autopct=lambda v: "{:.0f}".format(v/100 * len(df)))

In [ ]:
sns.scatterplot(data=df_1,
                x='ReadLength', y="Accuracy", hue="my_hue")

In [ ]:
sns.scatterplot(data=df_1,
                x='ReadLength', y="AlignmentScore", hue="my_hue")

### Question: What are those low accuracy reads?
    1 - Non-spike-in reads misassigned to Lambda
    2 - Chimeric reads with a lambda part and something else
    3 - Just poor reads

What if I calculate the alignment length in addition to the read length? Then I can also get AlignmentAccuracy, and I would predict:

    1 - These should have a low AlignmentAccuracy and the AlignmentLength should be << ReadLength
    2 - These should have a high AlignmentAccuracy and the AlignmentLength should be << ReadLength
    3 - These should have a low AlignmentAccuracy and the AlignmentLength should be ~= ReadLength


In [ ]:
# Almost the same but with AlignmentLength rather than alignmentscore. Much the same as above.
# Now plot, with three categories.

sns.scatterplot(data=df_1, x="ReadLength", y="AlignmentLength", hue="my_hue")


In [ ]:
# This reflect the straight line in the graph above

sns.scatterplot(data= df_1, x="AlignmentLength", y="AlignmentAccuracy", hue="my_hue")

OK, what questions do we have.
ReadLength vs. AligmnentLength shows that generally the whole read aligns but then we have a few things floating off the line and a long line at the top. My guess is that the top line is chimeric. Maybe the others are also chimeric but we have read part of Lambda plus part of whatever? Or I dunno.

AlignmentLength vs AlignmentAccuracy has most things floating at the top. So no relationship between them.

So now lets extract the outliers from the top graph. Stuff where the ReadLength - AlignmentLength > 10

In [ ]:
# Now, let's look at reads with the typical length.
df_1_inliers = df_1[df_1.my_hue == 'good']
sns.scatterplot(data=df_1_inliers,
                x='ReadLength', y="AlignmentLength", hue="AlignmentAccuracy")

In [ ]:
# So for these, what's the typical under-alignment? This is almost the previous graph upside-down, but it means we can
# read off how much is being trimmed in the BAM.

sns.scatterplot(data=df_1_inliers,
                x='ReadLength', y="UnalignedLength", hue="my_hue")
df_1_inliers.head()